# VacationPy
---

In [5]:
# Dependencies and Setup
import os
from dotenv import load_dotenv
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

import json
from pprint import pprint
import datetime
load_dotenv()

# Get geoapify api key
geoapify_key = os.getenv('geoapify_key')

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
0,0,codrington,-38.2667,141.9667,15.12,95,6,0.46,AU,2024-02-03
1,1,igarka,67.4667,86.5833,-11.93,96,100,4.78,RU,2024-02-03
2,2,wailua homesteads,22.0669,-159.3780,20.44,57,100,11.32,US,2024-02-03
3,3,waitangi,-43.9535,-176.5597,14.90,94,2,0.45,NZ,2024-02-03
4,4,altai,46.3722,96.2583,-14.58,96,100,1.35,MN,2024-02-03


## Create a map that displays a point for every city in the city_data_df DataFrame

In [7]:
%%capture --no-display

map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity (%)',
    color = 'City',
    frame_width = 700,
    frame_height = 500,
    xlabel = 'Longitude',
    ylabel = 'Latitude',
    title = 'Map of cities and humidity'
)

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%))

## Narrow down cities to find ideal weather condition for travel destination

In [8]:
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp (C)']>=24.00) &
                                    (city_data_df['Max Temp (C)']<=30.00) &
                                    (city_data_df['Humidity (%)']>=35) &
                                    (city_data_df['Humidity (%)']<=75) &
                                    (city_data_df['Cloudiness (%)']<=70) &
                                    (city_data_df['Wind Speed (m/s)']<=4.00)
                                   ]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (m/s),Country,Date
50,50,bamboo flat,11.7000,92.7167,28.05,69,20,2.06,IN,2024-02-03
83,83,kavaratti,10.5669,72.6420,26.75,68,70,2.46,IN,2024-02-03
115,115,cafayate,-26.0730,-65.9761,26.47,49,2,2.12,AR,2024-02-03
147,147,antofagasta,-23.6500,-70.4000,25.60,64,0,2.71,CL,2024-02-03
152,152,salinopolis,-0.6136,-47.3561,27.91,74,61,1.73,BR,2024-02-03
155,155,georgetown,5.4112,100.3354,29.97,73,20,0.51,MY,2024-02-03
169,169,brajarajnagar,21.8167,83.9167,24.05,43,0,2.06,IN,2024-02-03
203,203,santa cruz,-17.8000,-63.1667,29.03,70,0,2.57,BO,2024-02-03
208,208,port blair,11.6667,92.7500,26.41,69,17,3.95,IN,2024-02-03
277,277,aldinga beach,-35.2793,138.4604,25.14,66,0,3.84,AU,2024-02-03


## Create a map with the first hotel within 10km of cities with ideal weather for vacation

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# 2. https://saturncloud.io/blog/fastest-way-to-copy-columns-from-one-dataframe-to-another-using-pandas/
hotel_df = ideal_weather_df[['City','Country','Lat','Lng','Humidity (%)']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity (%),Hotel Name
50,bamboo flat,IN,11.7000,92.7167,69,
83,kavaratti,IN,10.5669,72.6420,68,
115,cafayate,AR,-26.0730,-65.9761,49,
147,antofagasta,CL,-23.6500,-70.4000,64,
152,salinopolis,BR,-0.6136,-47.3561,74,
155,georgetown,MY,5.4112,100.3354,73,
169,brajarajnagar,IN,21.8167,83.9167,43,
203,santa cruz,BO,-17.8000,-63.1667,70,
208,port blair,IN,11.6667,92.7500,69,
277,aldinga beach,AU,-35.2793,138.4604,66,


In [10]:
# For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.
# Set parameters to search for a hotel
radius = 10000
params = {'categories': 'accommodation.hotel',
          'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # 3. https://www.geeksforgeeks.org/pandas-dataframe-get_value/
    latitude = hotel_df._get_value(index, 'Lat')
    longitude = hotel_df._get_value(index, 'Lng')
 
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bamboo flat - nearest hotel: Hotel Megapode Nest
kavaratti - nearest hotel: No hotel found
cafayate - nearest hotel: Il Cavallino
antofagasta - nearest hotel: Hotel Colón
salinopolis - nearest hotel: Hotel Salinópolis
georgetown - nearest hotel: Page 63 hostel
brajarajnagar - nearest hotel: Hotel Orchard
santa cruz - nearest hotel: Aviador Hotel Boutique
port blair - nearest hotel: Andaman Ocean Breeze
aldinga beach - nearest hotel: The Beach Hotel
indapur - nearest hotel: No hotel found
kyaikkami - nearest hotel: ရှုခင်းသာ
bereeda - nearest hotel: No hotel found
port pirie - nearest hotel: Ellen Hotel
las khorey - nearest hotel: No hotel found
borba - nearest hotel: No hotel found
champerico - nearest hotel: Hotel y Restaurante El Submarino
mukalla - nearest hotel: فندق الجابري
manuguru - nearest hotel: No hotel found
boa vista - nearest hotel: Aipana Plaza Hotel


,City,Country,Lat,Lng,Humidity (%),Hotel Name
50,bamboo flat,IN,11.7000,92.7167,69,Hotel Megapode Nest
83,kavaratti,IN,10.5669,72.6420,68,No hotel found
115,cafayate,AR,-26.0730,-65.9761,49,Il Cavallino
147,antofagasta,CL,-23.6500,-70.4000,64,Hotel Colón
152,salinopolis,BR,-0.6136,-47.3561,74,Hotel Salinópolis
155,georgetown,MY,5.4112,100.3354,73,Page 63 hostel
169,brajarajnagar,IN,21.8167,83.9167,43,Hotel Orchard
203,santa cruz,BO,-17.8000,-63.1667,70,Aviador Hotel Boutique
208,port blair,IN,11.6667,92.7500,69,Andaman Ocean Breeze
277,aldinga beach,AU,-35.2793,138.4604,66,The Beach Hotel


In [11]:
%%capture --no-display

# Add the hotel name and the country as additional information in the hover message for each city in the map.
map_plot_ideal_destination = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = 'Humidity (%)',
    color = 'City',
    frame_width = 700,
    frame_height = 500,
    hover_cols = ['Hotel Name','Country'],
    xlabel = 'Longitude',
    ylabel = 'Latitude',
    title = 'My ideal travel destinations where humidity between 35-75%, max. temperature between 24-30C,\ncloudiness <= 70% and wind speed <= 4m/s'
)

# Display the map
map_plot_ideal_destination

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity (%),Hotel Name,Country)